<h2>The Task</h2>
Step 1:<br />
Download the data: close_price.csv, mv.csv<br />
Initial capital: 100 million CNY<br />
<br />
Step 2:<br />
For this step, using loop calculation is prohibited.<br />
Based on the data in mv.csv, select 100 stocks with largest mv everyday.<br />
Output the position matrix as a DataFrame (index: Dates, columns: Tickers).<br />
The method to calculate the position is: the holdings of each stock = 1% of the capital / the close_price of the stock.<br />
Hint: Because individual stocks are listed at different times, it is not true that all stocks have prices on all trading days. Please check this first.<br />
<br />
Step 3:<br />
Based on the result of step 2, output the list and number of stocks that need to be bought daily, and output the list and number of stocks that need to be sold daily.<br />
Hint: Using one day's mv.csv data to calculate the rank and the position change, one should conduct the transaction on the next day.<br />
<br />
Step 4:<br />
Based on the result of step 3, calculate daily profits.<br />
<br />
Step 5:<br />
Based on daily profits, calculate yearly total profit, winning rate, and turnover rate for each year.<br />
winning rate = number of days when the profit is negative / number of days in total.<br />
turnover rate = average daily transaction amount / total capital.<br />

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
# Step 1: Read the data
df_mv = pd.read_csv('./mv.csv')
df_mv = df_mv.rename(columns={'Unnamed: 0': 'date'})
df_mv['date'] = pd.to_datetime(df_mv['date'])
df_mv = df_mv.set_index('date')
df_close = pd.read_csv('./close_price.csv')
df_close = df_close.rename(columns={'Unnamed: 0': 'date'})
df_close['date'] = pd.to_datetime(df_close['date'])
df_close = df_close.set_index('date')

In [3]:
df_mv

,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,688686.SH,688687.SH,688689.SH,688696.SH,688698.SH,688699.SH,688777.SH,688788.SH,688819.SH,688981.SH
date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,1.577961e+11,2.638357e+11,1.348666e+09,2.826293e+09,6.952474e+09,2.813158e+09,1.093512e+10,9.242183e+09,2.689123e+09,5.560485e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,1.593414e+11,2.657124e+11,1.364621e+09,2.836879e+09,6.952474e+09,2.712688e+09,1.082239e+10,9.113223e+09,2.689123e+09,5.602203e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-04,1.674115e+11,2.752061e+11,1.394013e+09,2.910976e+09,7.100974e+09,2.771584e+09,1.101967e+10,9.392638e+09,2.721918e+09,5.697560e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-07,1.672398e+11,2.765308e+11,1.446078e+09,3.006245e+09,7.208974e+09,2.792371e+09,1.147060e+10,9.564585e+09,2.787506e+09,5.846555e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-08,1.658662e+11,2.759788e+11,1.411648e+09,2.995659e+09,7.181974e+09,2.719617e+09,1.166789e+10,9.715039e+09,2.762910e+09,5.804836e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-25,3.500828e+11,3.264583e+11,3.583292e+09,2.667513e+09,7.627472e+09,2.951737e+09,7.118836e+09,1.957623e+10,3.205632e+09,7.050433e+09,...,NaN,NaN,NaN,NaN,NaN,4.402586e+09,4.829381e+10,1.382276e+10,NaN,4.114180e+11
2020-12-28,3.658016e+11,3.301759e+11,3.375326e+09,2.635757e+09,7.424973e+09,2.944808e+09,6.979796e+09,1.895722e+10,2.943278e+09,6.841840e+09,...,NaN,NaN,NaN,NaN,NaN,4.157171e+09,5.028353e+10,1.105821e+10,NaN,4.006378e+11
2020-12-29,3.720115e+11,3.313377e+11,3.409987e+09,2.646342e+09,7.465473e+09,2.979453e+09,6.979796e+09,1.890564e+10,3.238426e+09,7.062352e+09,...,NaN,NaN,NaN,NaN,3.603600e+09,4.016616e+09,4.834785e+10,9.676690e+09,NaN,4.044879e+11


In [4]:
df_close

,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,688686.SH,688687.SH,688689.SH,688696.SH,688698.SH,688699.SH,688777.SH,688788.SH,688819.SH,688981.SH
date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,992.808557,3409.765152,65.265624,24.744500,176.264673,67.262247,86.425782,31.664856,35.340521,32.671542,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,1002.531383,3434.018712,66.037757,24.837176,176.264673,64.860024,85.534794,31.223021,35.340521,32.916666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-04,1053.306140,3556.713190,67.460109,25.485908,180.029549,66.268224,87.094022,32.180330,35.771503,33.476950,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-07,1052.225826,3573.833350,69.979704,26.319993,182.767641,66.765236,90.657972,32.769444,36.633467,34.352393,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-08,1043.583314,3566.699950,68.313520,26.227316,182.083118,65.025695,92.217200,33.284918,36.310230,34.107269,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-25,2003.318783,4324.609192,88.226444,23.354360,206.263149,70.575659,57.365230,67.410251,42.128487,44.373739,...,NaN,NaN,NaN,NaN,NaN,59.20,98.30,183.00,NaN,53.43
2020-12-28,2093.268240,4373.857411,83.105978,23.076331,200.787136,70.409988,56.244815,65.278702,38.680631,43.060906,...,NaN,NaN,NaN,NaN,NaN,55.90,102.35,146.40,NaN,52.03
2020-12-29,2128.803828,4389.247479,83.959389,23.169008,201.882339,71.238341,56.244815,65.101072,42.559469,44.448757,...,NaN,NaN,NaN,NaN,20.02,54.01,98.41,128.11,NaN,52.53


In [5]:
# Step 2: calculate the position
## Step 2.1: define selection functions
def quick_select(nums, k):
    pivot = random.choice(nums)
    left, equal, right = [], [], []
    for num in nums:
        if num < pivot:
            left.append(num)
        elif num > pivot:
            right.append(num)
        else:
            equal.append(num)
    len_right, len_equal = len(right), len(equal)
    if k >= len_right + 1 and k <= len_right + len_equal:
        return pivot
    if k <= len_right:
        return quick_select(right, k)
    return quick_select(left, k - len_right - len_equal)

def find_the_100th_largest(nums):
    return quick_select(nums.dropna(), 100)

## Step 2.2: do the calculation
df_position = 1000000 // df_close
the_100th_largest_mv = df_mv.apply(find_the_100th_largest, axis=1)
df_largest_100 = df_mv.apply(lambda x: np.where(x >= the_100th_largest_mv, 1, 0), axis=0)
df_position = df_position * df_largest_100
df_position.to_csv('my_position.csv')

In [6]:
df_position

,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,688686.SH,688687.SH,688689.SH,688696.SH,688698.SH,688699.SH,688777.SH,688788.SH,688819.SH,688981.SH
date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,1007.0,293.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,997.0,291.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-04,949.0,281.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-07,950.0,279.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-08,958.0,280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-25,499.0,231.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,18716.0
2020-12-28,477.0,228.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,19219.0
2020-12-29,469.0,227.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,19036.0


In [7]:
# Step 3: calculate the position change
df_pos_change = (df_position - df_position.shift()).shift()

In [8]:
df_pos_change

,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,688686.SH,688687.SH,688689.SH,688696.SH,688698.SH,688699.SH,688777.SH,688788.SH,688819.SH,688981.SH
date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-04,-10.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-07,-48.0,-10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-08,1.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-25,-7.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,432.0
2020-12-28,6.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,201.0
2020-12-29,-22.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,503.0


In [9]:
# Step 4: calculate daily profits
df_ret = ( - df_pos_change * df_close).sum(axis=1).to_frame(name="profit")

In [10]:
df_ret

,profit
date,
2019-01-02,0.000000e+00
2019-01-03,0.000000e+00
2019-01-04,1.300561e+05
2019-01-07,2.402166e+06
2019-01-08,4.443711e+05
...,...
2020-12-25,-2.188676e+05
2020-12-28,1.107654e+06
2020-12-29,3.835647e+05


In [11]:
# Step 5: calculate the annual total profit, winning rate, and turnover rate for each year
df_transaction_value = (abs(df_pos_change) * df_close).sum(axis=1).to_frame(name="transaction_value")
year_list, profit_list, winning_rate_list, turnover_rate_list = ['2019', '2020'], [], [], []
for year in year_list:
    profit = float(df_ret.loc[year].sum())
    profit_list.append(profit)
    winning_rate = int((df_ret.loc[year]>0).sum())/df_ret.loc[year].shape[0]
    winning_rate_list.append(winning_rate)
    turnover_rate = float(df_transaction_value.loc[year].mean()) / 100000000
    turnover_rate_list.append(turnover_rate)
df_result = pd.DataFrame({'year': year_list, 'profit': profit_list, 'winning_rate': winning_rate_list, 'turnover_rate': turnover_rate_list})
df_result.to_csv('my_result.csv')

In [12]:
df_result

,year,profit,winning_rate,turnover_rate
0,2019,3.924681e+07,0.536885,0.029691
1,2020,3.030716e+07,0.555556,0.040335


<h2>Commentary</h2>
This task focuses on three knowledge points:<br />
1. selection algorithm, particularly quickselect; see https://en.wikipedia.org/wiki/Quickselect  <br />
2. the application of df.apply; see https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html  <br />
3. the application of np.where; see https://numpy.org/doc/1.20/reference/generated/numpy.where.html  <br />
<br />
Among the five steps, the key step is step two. All the three knowledge points above are used in step two. If one used loop calculation, the calculation would be extremely slow. The runtime of using loop calculation would be at least 100 times (if not 1000 times) of the runtime of using the code here. In order to conduct the calculation fast, the three knowledge points above must be applied comprehensively.